In [1]:
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import pandas as pd
import h5py
from PIL import Image
from io import BytesIO


# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(False) # Needed for reproducible results

/data1/home/nidhi2023/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Random Seed:  999


In [2]:
# !kaggle competitions download -c isic-2024-challenge

In [3]:
# Setup device-agnostic code
device = "cuda:1" if torch.cuda.is_available() else "cpu"
device

'cuda:1'

In [4]:
train_matadata = pd.read_csv("train-metadata.csv", low_memory=False)
train_matadata.head()

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


In [5]:
class ImageLoader(Dataset):
    def __init__(self, df, file_hdf, transform=None):
        self.df = df
        self.fp_hdf = h5py.File(file_hdf, mode="r")
        self.isic_ids = df[df['target']!=1]['isic_id'].values
#         self.targets = df['target'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.isic_ids)
    
    def __getitem__(self, index):
        isic_id = self.isic_ids[index]
        image = Image.open(BytesIO(self.fp_hdf[isic_id][()]))
#         target = self.targets[index]
#         image = torch.permute(image,(2,1,0))
        if self.transform:
            return (self.transform(image))
        else:
            return (image)

In [6]:
train_transforms = transforms.Compose([
                               transforms.Resize(128),
                               transforms.CenterCrop(128),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])

train_dataset_full = ImageLoader(train_matadata,
                      file_hdf = "train-image.hdf5",
                      transform=train_transforms
                     )

train_size = int(0.95 * len(train_dataset_full))
test_size = len(train_dataset_full) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(train_dataset_full, [train_size, test_size])

In [7]:

import numpy as np


import torch
import torchvision
from torch import nn
from torch import autograd
from torch import optim

In [8]:
MODE = 'wgan-gp' # Valid options are dcgan, wgan, or wgan-gp
DIM = 32 # This overfits substantially; you're probably better off with 64
LAMBDA = 10 # Gradient penalty lambda hyperparameter
CRITIC_ITERS = 5 # How many critic iterations per generator iteration
BATCH_SIZE = 128 # Batch size
ITERS = 200000 # How many generator iterations to train for
OUTPUT_DIM = 3072 # Number of pixels in CIFAR10 (3*32*32)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        preprocess = nn.Sequential(
            nn.Linear(128, 2 * 2 * 4 * DIM),
            nn.BatchNorm1d(2 * 2 * 4 * DIM),
            nn.ReLU(True),
        )

        block1 = nn.Sequential(
            nn.ConvTranspose2d(4 * DIM, 4 * DIM, 2, stride=2),
            nn.BatchNorm2d(4 * DIM),
            nn.ReLU(True),
            nn.ConvTranspose2d(4 * DIM, 4 * DIM, 2, stride=2),
            nn.BatchNorm2d(4 * DIM),
            nn.ReLU(True),
            nn.ConvTranspose2d(4 * DIM, 4 * DIM, 2, stride=2),
            nn.BatchNorm2d(4 * DIM),
            nn.ReLU(True),
            nn.ConvTranspose2d(4 * DIM, 4 * DIM, 2, stride=2),
            nn.BatchNorm2d(4 * DIM),
            nn.ReLU(True),
            nn.ConvTranspose2d(4 * DIM, DIM, 2, stride=2),
            nn.BatchNorm2d(DIM),
            nn.ReLU(True),
        )
        deconv_out = nn.ConvTranspose2d(DIM, 3, 2, stride=2)

        self.preprocess = preprocess
        self.block1 = block1
        self.deconv_out = deconv_out
        self.tanh = nn.Tanh()

    def forward(self, input):
#         print(input.shape)
        output = self.preprocess(input)
#         print(output.shape)
        output = output.view(-1, 4 * DIM, 2, 2)
        output = self.block1(output)
#         print(output.shape)
        output = self.deconv_out(output)
#         print(output.shape)
        output = self.tanh(output)
        return output.view(-1, 3, 128, 128)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, DIM, 3, 2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(DIM, 2 * DIM, 3, 2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(2 * DIM, 4 * DIM, 3, 2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(4 * DIM, 6 * DIM, 3, 2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(6 * DIM, 3 * DIM, 3, 2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(3 * DIM, DIM, 3, 2, padding=1),
            nn.LeakyReLU(),
        )

        self.linear = nn.Linear(2*2*DIM, 1)

    def forward(self, input):
        output = self.main(input)
        output = output.view(-1, 2*2*DIM)
        output = self.linear(output)
        return output

netG = Generator()
netD = Discriminator()
print(netG)
print(netD)

use_cuda = torch.cuda.is_available()
if use_cuda:
    gpu = 0
if use_cuda:
    netD = netD.cuda(gpu)
    netG = netG.cuda(gpu)

Generator(
  (preprocess): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (block1): Sequential(
    (0): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)

In [9]:
one = torch.tensor(1, dtype=torch.float)
mone = one * -1
if use_cuda:
    one = one.cuda(gpu)
    mone = mone.cuda(gpu)

optimizerD = optim.Adam(netD.parameters(), lr=1e-4, betas=(0.5, 0.9))
optimizerG = optim.Adam(netG.parameters(), lr=1e-4, betas=(0.5, 0.9))

def calc_gradient_penalty(netD, real_data, fake_data):
#     print "real_data: ", real_data.size(), fake_data.size()
#     print(real_data.nelement())
    alpha = torch.rand(BATCH_SIZE, 1)
    alpha = alpha.expand(BATCH_SIZE, int(real_data.nelement()/BATCH_SIZE)).contiguous().view(BATCH_SIZE, 3, 128, 128)
    alpha = alpha.cuda(gpu) if use_cuda else alpha

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    if use_cuda:
        interpolates = interpolates.cuda(gpu)
    interpolates = autograd.Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).cuda(gpu) if use_cuda else torch.ones(
                                  disc_interpolates.size()),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty

# For generating samples
def generate_image(frame, netG):
    fixed_noise_128 = torch.randn(128, 128)
    if use_cuda:
        fixed_noise_128 = fixed_noise_128.cuda(gpu)
    with torch.no_grad():
        noisev = autograd.Variable(fixed_noise_128)#, volatile=True)
        samples = netG(noisev)
    samples = samples.view(-1, 3, 128, 128)
    samples = samples.mul(0.5).add(0.5)
    # samples = samples.cpu().data.numpy()
    torchvision.utils.save_image(samples,'./samples/samples_{}.jpg'.format(frame))
    # save_images(samples, './samples/samples_{}.jpg'.format(frame))


In [10]:
class InfiniteSampler(torch.utils.data.Sampler):
    def __init__(self, dataset, num_replicas=1, shuffle=True, seed=0):
        assert len(dataset) > 0
        super().__init__(dataset)
        self.dataset = dataset
        self.num_replicas = num_replicas
        self.shuffle = shuffle
        self.seed = seed

    def __iter__(self):
        order = np.arange(len(self.dataset))
        rnd = None
        window = 0
        if self.shuffle:
            rnd = np.random.RandomState(self.seed)
            rnd.shuffle(order)

        idx = 0
        while True:
            i = idx % order.size
            yield order[i]
            idx += 1

In [11]:
train_dataset[0].shape
training_set_sampler = InfiniteSampler(dataset=train_dataset)

dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                         sampler=training_set_sampler, num_workers=4,drop_last=True)
dev_gen = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=4,drop_last=True)



# Decide which device we want to run on
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

ds = iter(dataloader)


# next(ds).shape

In [12]:
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import collections
import time
import pickle

_since_beginning = collections.defaultdict(lambda: {})
_since_last_flush = collections.defaultdict(lambda: {})

_iter = [0]
def tick():
    _iter[0] += 1

def plot(name, value):
    _since_last_flush[name][_iter[0]] = value

def flush():
    prints = []

    for name, vals in _since_last_flush.items():
        prints.append("{}\t{}".format(name, np.mean(list(vals.values()))))
        _since_beginning[name] = vals
        x_vals = np.sort(list(_since_beginning[name].keys()))
        y_vals = [_since_beginning[name][x] for x in x_vals]

        plt.clf()
        plt.plot(x_vals, y_vals)
        plt.xlabel('iteration')
        plt.ylabel(name)
        plt.savefig(name.replace(' ', '_')+'.jpg')

    print ("iter {}\t{}".format(_iter[0], "\t".join(prints)))
    # _since_last_flush.clear()

    with open('log.pkl', 'wb') as f:
        pickle.dump(dict(_since_beginning), f, pickle.HIGHEST_PROTOCOL)
        
"""
Image grid saver, based on color_grid_vis from github.com/Newmu
"""

import numpy as np
import scipy.misc
# from keras.preprocessing.image import save_img
# from scipy.misc import imsave

def save_images(X, save_path):
    # [0, 1] -> [0,255]
    if isinstance(X.flatten()[0], np.floating):
        X = (255.99*X).astype('uint8')

    n_samples = X.shape[0]
    rows = int(np.sqrt(n_samples))
    while n_samples % rows != 0:
        rows -= 1

    nh, nw = rows, n_samples/rows

    if X.ndim == 2:
        X = np.reshape(X, (X.shape[0], int(np.sqrt(X.shape[1])), int(np.sqrt(X.shape[1]))))

    if X.ndim == 4:
        # BCHW -> BHWC
        X = X.transpose(0,2,3,1)
        h, w = X[0].shape[:2]
        img = np.zeros((int(h*nh), int(w*nw), 3))
    elif X.ndim == 3:
        h, w = X[0].shape[:2]
        img = np.zeros((h*nh, w*nw))
#     print(len(X),h,w)
#     print(img.shape)
    for n, x in enumerate(X):
        j = n//nw
        i = n%nw
#         print(j*h,i*w)
#         print(img[int(j*h):int(j*h+h), int(i*w):int(i*w+w)].shape)
        img[int(j*h):int(j*h+h), int(i*w):int(i*w+w)] = x

    torchvision.utils.save_image(img,save_path)
    


In [13]:
start_time = time.time()
for iteration in range(ITERS):
    
    ############################
    # (1) Update D network
    ###########################
    for p in netD.parameters():  # reset requires_grad
        p.requires_grad = True  # they are set to False below in netG update
        
    for i in range(CRITIC_ITERS):
        _data = next(ds)
        netD.zero_grad()

        # train with real
#         _data = _data.reshape(BATCH_SIZE, 3, 128, 128).permute(0, 2, 3, 1)
#         real_data = torch.stack([item for item in _data])
        real_data = _data

        if use_cuda:
            real_data = real_data.cuda(gpu)
        
        D_real = netD(real_data)
        D_real = D_real.mean()
        D_real.backward(mone)

        # train with fake
        noise = torch.randn(BATCH_SIZE, 128)
        if use_cuda:
            noise = noise.cuda(gpu)
            
        fake = netG(noise)
        D_fake = netD(fake)
        D_fake = D_fake.mean()
        D_fake.backward(one)
#         print(real_data_v.shape)

        # train with gradient penalty
        gradient_penalty = calc_gradient_penalty(netD, real_data.data, fake.data)
        gradient_penalty.backward()

        D_cost = D_fake - D_real + gradient_penalty
        Wasserstein_D = D_real - D_fake
        optimizerD.step()

    ############################
    # (2) Update G network
    ###########################
    for p in netD.parameters():
        p.requires_grad = False  # to avoid computation
    netG.zero_grad()

    noise = torch.randn(BATCH_SIZE, 128)
    if use_cuda:
        noise = noise.cuda(gpu)
    fake = netG(noise)
    G = netD(fake)
    G = G.mean()
    G.backward(mone)
    G_cost = -G
    optimizerG.step()

    # Write logs and save samples
    plot('./tmp/train disc cost', D_cost.cpu().data.numpy())
    plot('./tmp/time', time.time() - start_time)
    plot('./tmp/train gen cost', G_cost.cpu().data.numpy())
    plot('./tmp/wasserstein distance', Wasserstein_D.cpu().data.numpy())


    # Calculate dev loss and generate samples every 100 iters
    if iteration % 2000 == 0:
        print(time.time()-start_time)
        dev_disc_costs = []
        with torch.no_grad():
            for images in dev_gen:
                
                imgs = images
                # imgs = preprocess(images)
                if use_cuda:
                    imgs = imgs.cuda(gpu)
                imgs_v = autograd.Variable(imgs)

                D = netD(imgs_v)
                _dev_disc_cost = -D.mean().cpu().data.numpy()
                dev_disc_costs.append(_dev_disc_cost)
            plot('./tmp/dev disc cost', np.mean(dev_disc_costs))

            generate_image(iteration, netG)
    
    if (iteration % 4000 == 0):
        torch.save(netG.state_dict(), f'trained_nets/gen_{iteration}.pt')
        torch.save(netD.state_dict(), f'trained_nets/disc_{iteration}.pt')
    # Save logs every 100 iters
    if (iteration % 2000 == 0):
        flush()
    tick()

3.992314338684082
iter 0	./tmp/train disc cost	9.938619613647461	./tmp/time	3.9922008514404297	./tmp/train gen cost	0.05161835253238678	./tmp/wasserstein distance	0.008324846625328064	./tmp/dev disc cost	0.03966846689581871
1348.1284651756287
iter 2000	./tmp/train disc cost	-47.411014556884766	./tmp/time	698.497243160489	./tmp/train gen cost	-36.322757720947266	./tmp/wasserstein distance	70.22871398925781	./tmp/dev disc cost	-22.752971649169922
2571.138253211975
iter 4000	./tmp/train disc cost	-29.132654190063477	./tmp/time	1326.69594285304	./tmp/train gen cost	-24.03594398498535	./tmp/wasserstein distance	41.74999237060547	./tmp/dev disc cost	-25.417001724243164
3972.811932325363
iter 6000	./tmp/train disc cost	-22.296045303344727	./tmp/time	1967.0481881867288	./tmp/train gen cost	-15.211920738220215	./tmp/wasserstein distance	31.22220230102539	./tmp/dev disc cost	-21.40266227722168
5374.446241855621
iter 8000	./tmp/train disc cost	-18.542421340942383	./tmp/time	2639.2245919999145	./t